<a href="https://colab.research.google.com/github/MatthewK84/Crickett-Coding-Challenges/blob/main/Crontab_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import re

def is_valid_field(field, min_value, max_value):
    # Check for wildcards and steps like */5
    if field == '*' or re.match(r'^\*/\d+$', field):
        return True

    # Check for ranges and lists
    for part in field.split(','):
        if '-' in part:
            start, end = map(int, part.split('-'))
            if start > end or start < min_value or end > max_value:
                return False
        elif not part.isdigit() or not (min_value <= int(part) <= max_value):
            return False

    return True

def validate_cron_expression(expr):
    fields = expr.split()
    if len(fields) != 5:
        return False

    # Define min and max values for each field
    limits = [(0, 59), (0, 23), (1, 31), (1, 12), (0, 6)]

    return all(is_valid_field(field, *limit) for field, limit in zip(fields, limits))

def translate_field(field, field_name, min_value, max_value, names=None):
    if field == '*':
        return f"every {field_name}"

    if field.startswith('*/'):
        step = field.split('/')[1]
        return f"every {step} {field_name}s"

    parts = []
    for part in field.split(','):
        if '-' in part:
            start, end = part.split('-')
            parts.append(f"from {start} to {end} {field_name}s")
        else:
            value = int(part)
            name = names[value] if names and value in names else part
            parts.append(name)

    return ', '.join(parts)

def translate_cron_expression(expr):
    fields = expr.split()
    names_of_months = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June',
                       7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'}
    names_of_days = {0: 'Sunday', 1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 5: 'Friday', 6: 'Saturday'}

    minute = translate_field(fields[0], 'minute', 0, 59)
    hour = translate_field(fields[1], 'hour', 0, 23)
    day_of_month = translate_field(fields[2], 'day of the month', 1, 31)
    month = translate_field(fields[3], 'month', 1, 12, names_of_months)
    day_of_week = translate_field(fields[4], 'day of the week', 0, 6, names_of_days)

    return f"Minute: {minute}, Hour: {hour}, Day of the Month: {day_of_month}, Month: {month}, Day of the Week: {day_of_week}"

# User Interface
cron_expression = input("Enter a cron expression (e.g., '0 9 * * SAT'): ")

# Validation
if validate_cron_expression(cron_expression):
    # Translation
    translation = translate_cron_expression(cron_expression)
    print("Cron expression translated:", translation)
else:
    print("Invalid cron expression.")

Enter a cron expression (e.g., '0 9 * * SAT'): * * * * *
Cron expression translated: Minute: every minute, Hour: every hour, Day of the Month: every day of the month, Month: every month, Day of the Week: every day of the week
